<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Big Data II - Armazenamento
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Orientação a coluna;</li>
  <li>Particionamento.</li>
</ol>

---

# **Exercícios**

## 1\. Particionamento e orientação a coluna.

Replique as atividades da aula 3 em sua conta da AWS.

1.1. Faça o download do arquivo `crime.csv` (fonte [Kaggle](https://www.kaggle.com/yamqwe/chicago-crimee?select=chicago_crime_2014.csv)).

In [ ]:
!wget https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/crime.csv -q -O crime.csv

1.2. Leia os dados em um DataFrame Pandas e crie a coluna `reference_date`.

In [ ]:
import pandas as pd

filename = './crime'

df = pd.read_csv(f'./{filename}.csv')
df['reference_date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
#Pega a primeira parte do datetime
df['reference_date'] = df['Date'].str.split(' ').str[0]
df.head()

,index,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Latitude,Longitude,reference_date
0,0,9446824,HX100141,01/01/2014 02:00:00 AM,0000X W ILLINOIS ST,0460,BATTERY,SIMPLE,STREET,False,False,1831,18,42.0,8.0,08B,41.890828,-87.630235,01/01/2014
1,1,9446748,HX100020,01/01/2014 12:00:00 AM,006XX N DEARBORN ST,0890,THEFT,FROM BUILDING,BAR OR TAVERN,False,False,1832,18,42.0,8.0,06,41.893542,-87.629702,01/01/2014
2,2,9446758,HX100030,01/01/2014 12:30:00 AM,052XX W RACE AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,1523,15,28.0,25.0,14,41.890046,-87.756333,01/01/2014
3,3,9446760,HX100027,01/01/2014 12:30:00 AM,053XX W WELLINGTON AVE,0460,BATTERY,SIMPLE,APARTMENT,True,False,2514,25,31.0,19.0,08B,41.935011,-87.759739,01/01/2014
4,4,9446764,HX100054,01/01/2014 12:10:00 AM,014XX W LEXINGTON ST,0460,BATTERY,SIMPLE,STREET,False,False,1231,12,25.0,28.0,08B,41.872509,-87.663069,01/01/2014


In [ ]:
df.shape

(274423, 19)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274423 entries, 0 to 274422
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   index                 274423 non-null  int64  
 1   ID                    274423 non-null  int64  
 2   Case Number           274423 non-null  object 
 3   Date                  274423 non-null  object 
 4   Block                 274423 non-null  object 
 5   IUCR                  274423 non-null  object 
 6   Primary Type          274423 non-null  object 
 7   Description           274423 non-null  object 
 8   Location Description  274247 non-null  object 
 9   Arrest                274423 non-null  bool   
 10  Domestic              274423 non-null  bool   
 11  Beat                  274423 non-null  int64  
 12  District              274423 non-null  int64  
 13  Ward                  274421 non-null  float64
 14  Community Area        274422 non-null  float64
 15  

1.3. Persista o dado no DataFrame em um arquivo csv e em arquivos parquet, estes últimos comprimidos e particionados pela coluna `reference_date`.

In [ ]:
# Solução do item 1.3.

#criando csv
df.to_csv(f'./crime_dados_add.csv', index=False)

#compactando
df.to_parquet(f'./crime',
              engine='pyarrow',
              compression='gzip',
              partition_cols=['reference_date'],
              #adicionando o máximo de partições porque são muitas
              max_partitions=20000)


1.4. Crie os recursos no AWS S3 e AWS IAM e persista os dados em seus respectivos *buckets*.

In [ ]:
# Solução do item 1.4.

from getpass import getpass

print('Digite suas credenciais:')
print('\nchave id:')
aws_access_key_id = getpass()


Digite suas credenciais:

chave id:
··········


In [ ]:
print('chave secreta:')
aws_secret_access_key = getpass()

chave secreta:
··········


In [ ]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
import boto3

client = boto3.client(
  's3',
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key
)

In [ ]:
import os

BUCKET = 'modulo-41'

i = 0

#loop carregamento (visual)
for root, dirs, files in os.walk('./crime'):
  elapsed = f'{round(100*i/365, 2)} %'
  print(elapsed)

  #envio de arquivos ao bucket
  for file in files:
    path = os.path.join(root, file)
    bucket_path = '/'.join(path.split(sep='/')[2:])
    client.upload_file(path, BUCKET, bucket_path)
  i = i + 1


0.0 %
0.27 %
0.55 %
0.82 %
1.1 %
1.37 %
1.64 %
1.92 %
2.19 %
2.47 %
2.74 %
3.01 %
3.29 %
3.56 %
3.84 %
4.11 %
4.38 %
4.66 %
4.93 %
5.21 %
5.48 %
5.75 %
6.03 %
6.3 %
6.58 %
6.85 %
7.12 %
7.4 %
7.67 %
7.95 %
8.22 %
8.49 %
8.77 %
9.04 %
9.32 %
9.59 %
9.86 %
10.14 %
10.41 %
10.68 %
10.96 %
11.23 %
11.51 %
11.78 %
12.05 %
12.33 %
12.6 %
12.88 %
13.15 %
13.42 %
13.7 %
13.97 %
14.25 %
14.52 %
14.79 %
15.07 %
15.34 %
15.62 %
15.89 %
16.16 %
16.44 %
16.71 %
16.99 %
17.26 %
17.53 %
17.81 %
18.08 %
18.36 %
18.63 %
18.9 %
19.18 %
19.45 %
19.73 %
20.0 %
20.27 %
20.55 %
20.82 %
21.1 %
21.37 %
21.64 %
21.92 %
22.19 %
22.47 %
22.74 %
23.01 %
23.29 %
23.56 %
23.84 %
24.11 %
24.38 %
24.66 %
24.93 %
25.21 %
25.48 %
25.75 %
26.03 %
26.3 %
26.58 %
26.85 %
27.12 %
27.4 %
27.67 %
27.95 %
28.22 %
28.49 %
28.77 %
29.04 %
29.32 %
29.59 %
29.86 %
30.14 %
30.41 %
30.68 %
30.96 %
31.23 %
31.51 %
31.78 %
32.05 %
32.33 %
32.6 %
32.88 %
33.15 %
33.42 %
33.7 %
33.97 %
34.25 %
34.52 %
34.79 %
35.07 %
35.34 %
35.62 %
35

1.5. Crie os recursos no AWS Athena e execute as consultas SQL da aula.

In [ ]:
# Solução do item 1.5.

#Tabela

Criando tabela csv

```sql
CREATE EXTERNAL TABLE `crime_csv`(
  `index` bigint,
  `id` string,
  `case number` string,
  `date` string,
  `block` string,
  `iucr` string,
  `primary type` string,
  `description` string,
  `location description` string,
  `arrest` string,
  `domestic` string,
  `beat` string,
  `district` string,
  `ward` string,
  `community area` string,
  `fbi code` string,
  `latitude` string,
  `longitude` string,
  `reference_date` string)
ROW FORMAT SERDE
  'org.apache.hadoop.hive.serde2.OpenCSVSerde'
 WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '\"',
   'escapeChar' = '\\'
   )
STORED AS INPUTFORMAT
  'org.apache.hadoop.mapred.TextInputFormat'
OUTPUTFORMAT
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://modulo-41-planilha/'
TBLPROPERTIES (
  "skip.header.line.count"="1")
```

Criando tabela parquet

```sql
CREATE EXTERNAL TABLE `crime_parquet`(
  `index` bigint,
  `id` bigint,
  `case number` string,
  `date` string,
  `block` string,
  `iucr` string,
  `primary type` string,
  `description` string,
  `location description` string,
  `arrest` boolean,
  `domestic` boolean,
  `beat` bigint,
  `district` bigint,
  `ward` double,
  `community area` double,
  `fbi code` string,
  `latitude` double,
  `longitude` double)
PARTITIONED BY (
  `reference_date` string)
ROW FORMAT SERDE
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat'
OUTPUTFORMAT
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION
  's3://modulo-41/'
```

```sql
MSCK REPAIR TABLE `crime_parquet`;
```

#Seleção

###Seleção da tabela crime_csv:

```sql
SELECT "location description", COUNT(1) as "amount"
FROM crime_csv
GROUP BY 1
ORDER BY 2 DESC;
```

In [ ]:
!wget https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/descricao-local-csv.csv

--2024-10-08 16:26:51--  https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/descricao-local-csv.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3239 (3.2K) [text/plain]
Saving to: ‘descricao-local-csv.csv’

descricao-local-csv 100%[===================>]   3.16K  --.-KB/s    in 0s      

2024-10-08 16:26:51 (26.2 MB/s) - ‘descricao-local-csv.csv’ saved [3239/3239]



###Imagem seleção

![descricao-localizacao-csv](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Imagem/descricao-local-csv.png?raw=true)

###Seleção da tabela crime_parquet:

```sql
SELECT "location description", COUNT(1) as "amount"
FROM crime_parquet
GROUP BY 1
ORDER BY 2 DESC;
```

In [ ]:
!wget https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/descricao-local-parquet.csv

--2024-10-08 16:27:00--  https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/descricao-local-parquet.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3237 (3.2K) [text/plain]
Saving to: ‘descricao-local-parquet.csv’

descricao-local-par 100%[===================>]   3.16K  --.-KB/s    in 0s      

2024-10-08 16:27:00 (44.7 MB/s) - ‘descricao-local-parquet.csv’ saved [3237/3237]



###Imagem seleção

![descricao-localizacao-parquet](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Imagem/descricao-local-parquet.png?raw=true)

###Seleção da tabela crime_csv:

```sql
SELECT * FROM crime_csv
WHERE CAST(date_parse(replace(reference_date, '-', '/'), '%m/%d/%Y') as DATE)
BETWEEN DATE '2014-12-01' and DATE '2014-12-31';
```

In [ ]:
!wget https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/tabela-csv-filtrada.csv

--2024-10-08 16:28:19--  https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/tabela-csv-filtrada.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4567925 (4.4M) [text/plain]
Saving to: ‘tabela-csv-filtrada.csv’

tabela-csv-filtrada 100%[===================>]   4.36M  --.-KB/s    in 0.05s   

2024-10-08 16:28:19 (80.6 MB/s) - ‘tabela-csv-filtrada.csv’ saved [4567925/4567925]



###Tempo de execução:

![tempo-execucao-csv](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Imagem/tempo-execucao-csv-dado-completo.png?raw=true)

###Seleção da tabela crime_parquet:

```sql
SELECT * FROM crime_parquet
WHERE CAST(date_parse(replace(reference_date, '-', '/'), '%m/%d/%Y') as DATE)
BETWEEN DATE '2014-12-01' and DATE '2014-12-31';
```

In [ ]:
!wget https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/tabela-parquet-filtrada.csv

--2024-10-08 16:28:56--  https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/tabela-parquet-filtrada.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4565279 (4.4M) [text/plain]
Saving to: ‘tabela-parquet-filtrada.csv’

tabela-parquet-filt 100%[===================>]   4.35M  --.-KB/s    in 0.06s   

2024-10-08 16:28:56 (69.4 MB/s) - ‘tabela-parquet-filtrada.csv’ saved [4565279/4565279]



###Tempo de execução:

![tempo-execucao-parquet](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Imagem/tempo-execucao-parquet-dado-completo.png?raw=true)

###Seleção da tabela crime_csv:

```sql
SELECT "location description", COUNT(1) as "amount"
FROM crime_csv
WHERE CAST(date_parse(replace(reference_date, '-', '/'), '%m/%d/%Y') as DATE)
BETWEEN DATE '2014-12-01' and DATE '2014-12-31'
GROUP BY 1
ORDER BY 2 DESC;
```

In [ ]:
!wget https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/tabela-csv-filtrada-2-colunas.csv

--2024-10-08 16:29:45--  https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/tabela-csv-filtrada-2-colunas.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2694 (2.6K) [text/plain]
Saving to: ‘tabela-csv-filtrada-2-colunas.csv’

tabela-csv-filtrada 100%[===================>]   2.63K  --.-KB/s    in 0s      

2024-10-08 16:29:45 (22.8 MB/s) - ‘tabela-csv-filtrada-2-colunas.csv’ saved [2694/2694]



###Tempo de execução:

![tempo-execucao-csv](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Imagem/tempo-execucao-csv-duas-colunas.png?raw=true)

###Seleção da tabela crime_parquet:

```sql
SELECT "location description", COUNT(1) as "amount"
FROM crime_parquet
WHERE CAST(date_parse(replace(reference_date, '-', '/'), '%m/%d/%Y') as DATE)
BETWEEN DATE '2014-12-01' and DATE '2014-12-31'
GROUP BY 1
ORDER BY 2 DESC;
```

In [ ]:
!wget https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/tabela-parquet-filtrada-2-colunas.csv

--2024-10-08 16:29:58--  https://raw.githubusercontent.com/mariaefoliveira/Analise-de-Dados/refs/heads/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Planilhas/tabela-parquet-filtrada-2-colunas.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692 (2.6K) [text/plain]
Saving to: ‘tabela-parquet-filtrada-2-colunas.csv’

tabela-parquet-filt 100%[===================>]   2.63K  --.-KB/s    in 0s      

2024-10-08 16:29:58 (40.1 MB/s) - ‘tabela-parquet-filtrada-2-colunas.csv’ saved [2692/2692]



###Tempo de execução:

![tempo-execucao-parquet](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2041%20-%20Ebac/Imagem/tempo-execucao-parquet-duas-colunas.png?raw=true)

# Conclusão

Ao comparar o tempo de execução da consulta de uma tabela **csv** e com o **parquet**, foi perceptível que o do **parquet** teve uma execução bem mais rápida, tendo o tamanho de dados verificado bem menor do que o do **csv**

Ocasionando numa execução mais rápida e que exige menos processamento.